In [1]:
from threeDprediction import *
from threeDprediction_multi import *
from pore_analysis import *
from visualization import *
import os
import numpy as np
from skimage import io

ModuleNotFoundError: No module named 'segmentation_models_3D'

# Welcome to the Jupyter Notebook for the utilization of UTILE-Pore

With this notebook, you will be guided through the code and make it possible for you to analyze your porous materials tomographies automatically.

This notebook should help you to run the automated segmentation of the volumes in your images and afterward to apply the diverse functions to extract the information of interest from your data and visualize the results.

Already integrated functions are:

- GDL/porous material
- Pore size distribution analysis of the pores with PoreSpy
- Surface roughness calculation
- Tortuosity simulation with PoreSpy
- Permeability estimation using Kozeny-Carman equation
- Calculation of the solid surface ratio


For MPL
- Layer thickness calculation
- MPL crack analsis (ratio, size distribution, etc.)
- MPL heatmap for thickness variation
- MPL intrusion measurement
- MPL and GDL contact surface calculation

In [ ]:
#Give a name to your project

case_name = "example1"

#First, we need to specify the folder where the tomograph slices (or tif stack) are stored

image_path= "./path/to/image.tif"

#Secondly, we need to specify where is your model stored (.hdf5)

model_path = "./path/to/model.hdf5"

# We set the path to the csv file to store all the extracted data

csv_file = './functions/csv_test.csv'

#It is also required to create a folder to store the predicted masks

os.makedirs(f"./{case_name}/mask_folder", exist_ok=True)
mask_folder = f"./{case_name}/mask_folder/"

In [ ]:
# Now we start with the segmentation of the volume
# If you just have a porous materials and need a binary segmentation, run the following lines

# input_volume = io.imread('/p/project1/claimd/Andre/Aimy/Dataset/cal_fusev2/test/Toray1202.tif')
# predicted_volume = process_and_predict(image_path, model_path)
# io.imsave(f'./predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

#If you are analyzing a GDL with MPL, then you can run a multiclass semantic segmentation with the following lines:
input_volume = io.imread('/p/project1/claimd/Andre/Aimy/Dataset/cal_fusev2/test/Toray1202.tif')
predicted_volume = process_and_predict_multi(input_volume)
io.imsave(f'./predicted_{case_name}.tif', predicted_volume.astype(np.uint8))

In [ ]:
# Now, lets visualize the segmented volume
predicted_volume = io.imread('/p/project1/claimd/Andre/Aimy/Dataset/cal_fusev2/test/Toray1202.tif')
visualize_volume(predicted_volume, case_name, False)

In [ ]:
# We can directly start wiht the property extraction of the porous material or GDL
# We start with the pore size distribution analysis

porosity, results, avg_pore, sd = calculate_psd(predicted_volume, csv_file, voxel_size=5)

In [ ]:
#Tortuosity simulation (for non-optimized devices can take several hours)
#tortuosity = 1.85
tortuosity = tortuosity_simulation(predicted_volume, csv_file)

In [ ]:
# Having the above mentioned properties, we can estimate the permeability
# Estimate permeability
ssa = calculate_ssa(predicted_volume)
permeability = estimate_permeability(porosity, csv_file, ssa, tortuosity)
print(f'Permeability: {permeability}')

In [ ]:
# Then surface roughness calculation of the material
#Calculate surface roughness
Ra, Rq = calculate_surface_roughness_from_surface(predicted_volume)
#MPL_roughness_calculation(binary_image_3d, mpl=2)
print(f"Arithmetic Mean Roughness (Ra): {Ra}")
print(f"Root Mean Square Roughness (Rq): {Rq}")


In [ ]:
#Calculacte solid surface ratio
calculate_solid_surface_ratio(predicted_volume, csv_file, gdl=1, side='bottom')

In [ ]:
# Now we can move to the functions specialized for MPL containing GDLs
# We can caluclate the MPL and GDL thickness
MPL_GDL_thickness(predicted_volume, csv_file, axis=0, mpl=2, gdl=1)


In [ ]:
#Calculate MPL crack analysis
crack_ratio, crack_count, crack_labels, crack_sizes, slice_image = MPL_crack_analysis(predicted_volume, csv_file)
print(f"Crack Ratio: {crack_ratio}")
print(f"Crack Count: {crack_count}")
print(f"Crack Sizes: {crack_sizes}")
plot_crack_labels(slice_image, crack_labels)


In [ ]:
# Example usage:
Ra, Rq, Ra_std_dev, Ra_CoV, avg_thickness = MPL_intrusion_roughness(predicted_volume, csv_file, mpl=2, voxel_size=5)
print(f"Global Roughness Ra: {Ra}, Rq: {Rq}")
print(f"Standard Deviation of Local Ra: {Ra_std_dev}")
print(f"Coefficient of Variation of Local Ra: {Ra_CoV}")
print(f"Average Thickness: {avg_thickness}")

In [ ]:
# MPL heatmap creation
MPL_heatmap(predicted_volume,2)

In [ ]:
# Calculate fiber touching MPL voxels
MPL_count_touching_voxels(predicted_volume, csv_file, mpl_class=1, fiber_class=2)